In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
import glob

# # 멕북용 폰트 불러오기 무시해주세용
# font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf'
# font = fm.FontProperties(fname=font_path).get_name()
# matplotlib.rc('font', family=font)

In [108]:
# 푸드 데이터 로드
path_food = "./data/EC_SNS_KFOOD_ATTRACTION_DATA_*.csv"
files_food = glob.glob(path_food)
df_food = [pd.read_csv(f, encoding='utf-8') for f in files_food]
sns_kfood = pd.concat(df_food, ignore_index=True)

# 뷰티 데이터 로드
path_beauty = "./data/EC_SNS_KBEAUTY_ATTRACTION_DATA_*.csv"
files_beauty = glob.glob(path_beauty)
df_beauty = [pd.read_csv(f, encoding='utf-8') for f in files_beauty]
sns_beauty = pd.concat(df_beauty, ignore_index=True)

# 데이터 병합
full_df = pd.concat([sns_kfood, sns_beauty], ignore_index=True)

### 필요한 컬럼명 정리
#### 그대로 readme에 올려도 될듯합니다
|컬럼영문명|컬럼한글명|필요 컬럼|
|---|---|---|
|TRRSRT_NM|관광지명||
|BASE_YM|기준년월|O|
|CHNNEL_NM|채널명||
|BASE_YEAR_ACCMLT_FQ_CO|기준년도누적빈도수||
|BASE_YM_FQ_CO|	기준년월빈도수||
|BASE_YEAR_BEFORE_MT_FQ_CO|기준년도이전월빈도수||
|BEFORE_MT_VERSUS_FQ_CO_IRDS_RT|이전월대비빈도수증감율||
|AVRG_SCORE_VALUE|평점값|O|
|REVIEW_CO|리뷰수|O|
|PLACE_TY|장소유형||
|LC_LA|위치위도||
|LC_LO|위치경도||
|ADDR|주소||
|CTPRVN_NM|시도명|O|
|SIGNGU_NM|시군구명|O|
|TURSM_CSTMR_CO|관광고객수|O|
|BEFORE_YEAR_MT_TURSM_CSTMR_CO|이전년도월관광고객수||
|TURSM_CSTMR_CO_IRDS_RT|관광고객수증감율||
|TURSM_SPND_PRICE|관광소비금액|O|
|NATIVE_TURSM_SPND_PRICE|내국인관광소비금액|O|
|FRNR_TURSM_SPND_PRICE|외국인관광소비금액|O|

In [109]:
# 사용할 컬럼 추출
use_cols = ['BASE_YM', 'CTPRVN_NM', 'SIGNGU_NM', 'TURSM_CSTMR_CO', 
            'TURSM_SPND_PRICE', 'NATIVE_TURSM_SPND_PRICE', 
            'FRNR_TURSM_SPND_PRICE', 'AVRG_SCORE_VALUE', 'REVIEW_CO'] 
df_sub = full_df[use_cols].copy()

# 데이터 형 변환
df_sub['BASE_YM'] = pd.to_datetime(df_sub['BASE_YM'], format='%Y%m')
df_sub = df_sub.fillna(0)

# 통합 데이터 셋 (뷰티는 구별로 다른 구간도 있고 같은 구간도 있어서 평균으로 처리함)
final_df = df_sub.groupby(['BASE_YM', 'CTPRVN_NM', 'SIGNGU_NM']).agg({
    'TURSM_SPND_PRICE': 'mean',          # 관광소비금액
    'TURSM_CSTMR_CO': 'mean',            # 관광고객수
    'NATIVE_TURSM_SPND_PRICE': 'mean',   # 내국인관광소비금액
    'FRNR_TURSM_SPND_PRICE': 'mean',     # 외국인관광소비금액
    'AVRG_SCORE_VALUE': 'mean',          # 평점값
    'REVIEW_CO': 'mean'                  # 리뷰수
}).reset_index()

In [110]:
# pct_change(): 전년대비 변화량을 계산해 주는 함수 <(현재값 - 이전값) / 이전값>
# (현재값 - 이전값) / 이전값 * 100 = 현재 성장률
final_df = final_df.sort_values(['SIGNGU_NM', 'BASE_YM'])   # pct_change()를 쓰려면 정렬이 필요
final_df = final_df.reset_index(drop=True)

# 몇몇개의 시군구명이 KR로 들어가 있는 데이터랑 '0' 들어가 있는 데이터 제거
final_df = final_df[~final_df['SIGNGU_NM'].isin(['KR', '0', 0])]
final_df = final_df.reset_index(drop=True)

# 전월 대비 소비 증감률 = (현재달 - 지난달) / 지난달 * 100
final_df['SPND_GROWTH_RATE'] = final_df.groupby('SIGNGU_NM')['TURSM_SPND_PRICE'].pct_change() * 100

# 관광고객수 증감률
final_df['CSTMR_GROWTH_RATE'] = final_df.groupby('SIGNGU_NM')['TURSM_CSTMR_CO'].pct_change() * 100

# 리뷰수 증감률
final_df['REVIEW_GROWTH_RATE'] = final_df.groupby('SIGNGU_NM')['REVIEW_CO'].pct_change() * 100


# 결측치 및 무한대를 이전 값으로 처리 
# 변화률 위주이기 때문에 0을 넣어서 이전달과 변화가 없다 라고 평가하는게 맞다고 봄
final_df = final_df.replace([np.inf, -np.inf], np.nan)
final_df = final_df.fillna(0)


# 이상치 제거
# 중간에 보면 이상치가 8000 까지 뛸때가 있음
# 실측 오류이거나 데이터 등록에서 오류가 발생했다고 보고 마찬가지로 변화가 없다는 의미의 0으로 처리

# 소비 증감률이 500%를 넘는 경우 0으로 처리
final_df.loc[abs(final_df['SPND_GROWTH_RATE']) > 500, 'SPND_GROWTH_RATE'] = 0

# 고객수 증감률이 500%를 넘는 경우 0으로 처리
final_df.loc[abs(final_df['CSTMR_GROWTH_RATE']) > 500, 'CSTMR_GROWTH_RATE'] = 0

# 리뷰수 증감률이 500%를 넘는 경우 0으로 처리
final_df.loc[abs(final_df['REVIEW_GROWTH_RATE']) > 500, 'REVIEW_GROWTH_RATE'] = 0

final_df

,BASE_YM,CTPRVN_NM,SIGNGU_NM,TURSM_SPND_PRICE,TURSM_CSTMR_CO,NATIVE_TURSM_SPND_PRICE,FRNR_TURSM_SPND_PRICE,AVRG_SCORE_VALUE,REVIEW_CO,SPND_GROWTH_RATE,CSTMR_GROWTH_RATE,REVIEW_GROWTH_RATE
0,2025-02-01,경기도,가평군,4526283.000,1330686.000,4356036.000,170247.000,4.400,36432.000,0.000,0.000,0.000
1,2025-07-01,경기도,가평군,655573.000,2688308.000,285133.000,370440.000,4.400,36800.000,-85.516,102.024,1.010
2,2025-08-01,경기도,가평군,56729443.000,3322011.000,56258056.000,471387.000,4.400,36877.000,0.000,23.573,0.209
3,2025-09-01,경기도,가평군,39090303.000,2051913.000,38434403.000,655900.000,4.400,30306.667,-31.093,-38.233,-17.817
4,2025-10-01,경기도,가평군,48143693.000,2644120.000,47364911.000,778782.000,4.400,30416.667,23.160,28.861,0.363
...,...,...,...,...,...,...,...,...,...,...,...,...
855,2025-11-01,강원특별자치도,홍천군,33648648.000,1640793.000,33445264.000,203384.000,4.300,6697.000,-22.304,-26.987,0.030
856,2025-12-01,강원특별자치도,홍천군,36272116.000,1632712.000,34719603.000,1552513.000,4.300,6705.000,7.797,-0.493,0.119
857,2025-03-01,서울특별시,화양동,37341094.000,6797980.000,22373501.000,14967593.000,4.100,812.000,0.000,0.000,0.000
858,2025-04-01,서울특별시,화양동,33977729.000,5887682.000,21736852.000,12240877.000,4.100,817.000,-9.007,-13.391,0.616


### 외국인 관광소비금액 예측
예측 타깃
외국인 관광 소비금액
외국인 관광 소비 예측”은 정책·마케팅 활용성 명확
1 수요 관련

TURSM_CSTMR_CO

NATIVE_TURSM_SPND_PRICE

이전월 외국인 소비금액 (lag)

2 지역

CTPRVN_NM

SIGNGU_NM

3 인지도/매력

AVRG_SCORE_VALUE

REVIEW_CO

REVIEW_GROWTH_RATE

4 시계열

월 (BASE_YM → month)

성수기 더미

모델
Linear Regression (baseline)

RandomForestRegressor

GradientBoosting / XGBoost

평가
RMSE / MAE

지역별 오차 비교

In [111]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.impute import SimpleImputer

pd.options.display.float_format = '{:.3f}'.format # 소수점 3자리까지만 표시

In [112]:
df = final_df.copy() # 복사본
# 날짜 처리
df['BASE_YM'] = pd.to_datetime(df['BASE_YM'])

# 지역 키
group_keys = ['CTPRVN_NM', 'SIGNGU_NM']

# 월 파생
df["year"] = df["BASE_YM"].dt.year
df["month"] = df["BASE_YM"].dt.month
df["quarter"] = df["BASE_YM"].dt.quarter

# 나눗셈 방지
eps = 1e-9
df['frnr_share_spend'] = df['FRNR_TURSM_SPND_PRICE'] / (df['TURSM_SPND_PRICE'] + eps)
df['frnr_share_spend'] = df['frnr_share_spend'].clip(0, 1)

# 이동평균 및 이동표준편차 피처 생성
for col in ["FRNR_TURSM_SPND_PRICE", "TURSM_CSTMR_CO", "REVIEW_CO"]:
    g = df.groupby(group_keys)[col]
    df[f"{col}_roll3_mean"] = g.shift(1).rolling(3, min_periods=1).mean()
    df[f"{col}_roll3_std"]  = g.shift(1).rolling(3, min_periods=2).std()


# 타겟 변수

# 학습에 쓰지 않을 컬럼
drop_cols = ['FRNR_TURSM_SPND_PRICE']

# 카테고리/수치 피처 구분
categorical_features = ["CTPRVN_NM", "SIGNGU_NM"]  # 지역
exclude_cols = ["frnr_share_spend"] # 누수 변수


numeric_features = [c for c in df.columns
                    if c not in (drop_cols + exclude_cols + ["BASE_YM", "CTPRVN_NM", "SIGNGU_NM"])]

X = df[categorical_features + numeric_features]
y = df['FRNR_TURSM_SPND_PRICE'].clip(0, None)  # 음수 방지

# 테스트 데이터 분리
# train_test_split 안쓴이유는 시계열 데이터이기 때문 -> 시간기반
last_date = df["BASE_YM"].max()
test_start = (last_date - pd.offsets.MonthBegin(2))  # 대략 마지막 2개월
train_idx = df["BASE_YM"] < test_start
test_idx  = df["BASE_YM"] >= test_start

X_train, y_train = X.loc[train_idx], y.loc[train_idx]
X_test,  y_test  = X.loc[test_idx],  y.loc[test_idx]
# 전처리 파이프라인

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

In [113]:
# 모델 선택

ridge = Ridge(alpha=1.0, random_state=42)

# 트리 기반 
hgb = HistGradientBoostingRegressor(
    learning_rate=0.05,
    max_depth=5,
    max_iter=500,
    random_state=42
)

# 랜덤 포레스트

rf_clf = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=5,
    n_jobs=-1,
    random_state=42
)

model = {
    'Ridge' : ridge,
    'HistGB' : hgb,
    'RF' : rf_clf
}

In [114]:
def eval(m_n, model, X_train, y_train, X_test, y_test):
    # 파이프라인 생성
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    pipe.fit(X_train, y_train)
    
    y_pred = pipe.predict(X_test)
    
    pred = np.clip(y_pred, 0, None)
    true = y_test.to_numpy()
    
    mae = mean_absolute_error(true, pred)
    rmse = root_mean_squared_error(true, pred)
    mape = np.mean(np.abs((true - pred) / (true + 1e-9))) * 100  # MAPE 계산 -> 지표 해석 쉽게 하기 위해 넣음
    
    print(f"{m_n} - MAE: {mae:,.0f}, RMSE: {rmse:,.0f}, MAPE: {mape:.2f}%")
    return pipe, {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

fitted_models = {}
scores = {}

for m_n, m in model.items():
    pipe, score = eval(m_n, m, X_train, y_train, X_test, y_test)
    fitted_models[m_n] = pipe
    scores[m_n] = score

print(f'모델 평가 결과 {scores}')


Ridge - MAE: 8,900,748, RMSE: 13,641,832, MAPE: 463.72%
HistGB - MAE: 6,071,129, RMSE: 11,620,189, MAPE: 198.43%
RF - MAE: 7,106,247, RMSE: 17,149,379, MAPE: 120.42%
모델 평가 결과 {'Ridge': {'MAE': 8900748.082832698, 'RMSE': 13641831.863827325, 'MAPE': np.float64(463.71950064271437)}, 'HistGB': {'MAE': 6071129.120474522, 'RMSE': 11620189.41040099, 'MAPE': np.float64(198.43049850310902)}, 'RF': {'MAE': 7106247.023154856, 'RMSE': 17149378.779131547, 'MAPE': np.float64(120.4187303596122)}}


In [117]:
# 지역별 예측 결과 샘플 확인
sample_name = pd.DataFrame(scores).T.sort_values('RMSE').index[0]
best_model = fitted_models[sample_name]

test_out = df.loc[test_idx, ["BASE_YM", "CTPRVN_NM", "SIGNGU_NM", "FRNR_TURSM_SPND_PRICE"]].copy()
pred_amt = best_model.predict(X_test)
test_out["PRED_FRNR_TURSM_SPND_PRICE"] = np.clip(pred_amt, 0, None)  # 원 스케일
test_out.sort_values('PRED_FRNR_TURSM_SPND_PRICE', ascending=False).head(10)

,BASE_YM,CTPRVN_NM,SIGNGU_NM,FRNR_TURSM_SPND_PRICE,PRED_FRNR_TURSM_SPND_PRICE
711,2025-10-01,서울특별시,중구,237993945.000,233058043.738
715,2025-11-01,서울특별시,중구,258909107.000,230072004.858
719,2025-12-01,서울특별시,중구,271894598.000,230072004.858
20,2025-11-01,서울특별시,강남구,254136281.000,206536079.968
19,2025-10-01,서울특별시,강남구,241550907.000,205176223.654
21,2025-12-01,서울특별시,강남구,263516405.000,191755568.802
389,2025-12-01,서울특별시,서초구,75384874.000,104242364.753
388,2025-12-01,서울시,서초구,75384874.000,89255320.246
387,2025-11-01,서울특별시,서초구,67956635.000,88866046.984
712,2025-10-01,인천광역시,중구,65867101.000,87231670.367
